In [1]:
import pandas as pd
import numpy as np

In [5]:
csv_file_path = r'C:\Users\gijsv\OneDrive\Documenten\Ironhack\week3\project\Data\Education_spending\share-of-education-in-government-expenditure.csv'
df = pd.read_csv(csv_file_path)
df = df[df['Year'] >= 1980]
df['Year'].unique()

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 1996, 1988, 1989, 1990, 1991, 1992, 1993, 1994,
       1995, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987], dtype=int64)

In [6]:
df

,Entity,Code,Year,"Government expenditure on education, total (% of government expenditure)"
0,Afghanistan,AFG,2005,15.080000
1,Afghanistan,AFG,2006,12.880000
2,Afghanistan,AFG,2007,12.360000
3,Afghanistan,AFG,2008,16.650000
4,Afghanistan,AFG,2009,17.310000
...,...,...,...,...
4099,Zimbabwe,ZWE,2015,29.470830
4100,Zimbabwe,ZWE,2016,23.527080
4101,Zimbabwe,ZWE,2017,20.874200
4102,Zimbabwe,ZWE,2018,19.039840
